In [1]:
import nltk
nltk.download('punkt')
import urllib.request

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

케라스에서 제공하는 스팸 메일 데이터 사용<br>
https://www.kaggle.com/uciml/sms-spam-collection-dataset

In [59]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv',encoding='latin1')

In [60]:
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


Unnamed2, Unnamed3, Unnaed4 제거

In [61]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']
data['v1']=data['v1'].replace(['ham', 'spam'], [0, 1])

In [62]:
data

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


데이터 전처리

1) 대소문자 변경(정제 과정)

In [63]:
for i in range(data.shape[0]):
  data['v2'][i]=data['v2'][i].lower()
data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,v1,v2
0,0,"go until jurong point, crazy.. available only ..."
1,0,ok lar... joking wif u oni...
2,1,free entry in 2 a wkly comp to win fa cup fina...
3,0,u dun say so early hor... u c already then say...
4,0,"nah i don't think he goes to usf, he lives aro..."
...,...,...
5567,1,this is the 2nd time we have tried 2 contact u...
5568,0,will ì_ b going to esplanade fr home?
5569,0,"pity, * was in mood for that. so...any other s..."
5570,0,the guy did some bitching but i acted like i'd...


2) 구두점 제거(정제 과정)

In [64]:
import re
for i in range(data.shape[0]):
  data['v2'][i]=re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》;]', '', data['v2'][i])
data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,v1,v2
0,0,go until jurong point crazy available only in ...
1,0,ok lar joking wif u oni
2,1,free entry in 2 a wkly comp to win fa cup fina...
3,0,u dun say so early hor u c already then say
4,0,nah i dont think he goes to usf he lives aroun...
...,...,...
5567,1,this is the 2nd time we have tried 2 contact u...
5568,0,will ì_ b going to esplanade fr home
5569,0,pity was in mood for that soany other suggest...
5570,0,the guy did some bitching but i acted like id ...


3) 길이가 2 이하인 단어 제거(정제 과정)

In [65]:
#import re
for i in range(data.shape[0]):
  shortword = re.compile(r'\W*\b\w{1,2}\b')
  data['v2'][i]=shortword.sub('', data['v2'][i])
data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,v1,v2
0,0,until jurong point crazy available only bugis...
1,0,lar joking wif oni
2,1,free entry wkly comp win cup final tkts 21st m...
3,0,dun say early hor already then say
4,0,nah dont think goes usf lives around here though
...,...,...
5567,1,this the 2nd time have tried contact have won ...
5568,0,will going esplanade home
5569,0,pity was mood for that soany other suggestions
5570,0,the guy did some bitching but acted like inter...


4) 단어 토큰화

In [66]:
from nltk.tokenize import word_tokenize
for i in range(data.shape[0]):
  data['v2'][i]=word_tokenize(data['v2'][i])
data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,v1,v2
0,0,"[until, jurong, point, crazy, available, only,..."
1,0,"[lar, joking, wif, oni]"
2,1,"[free, entry, wkly, comp, win, cup, final, tkt..."
3,0,"[dun, say, early, hor, already, then, say]"
4,0,"[nah, dont, think, goes, usf, lives, around, h..."
...,...,...
5567,1,"[this, the, 2nd, time, have, tried, contact, h..."
5568,0,"[will, going, esplanade, home]"
5569,0,"[pity, was, mood, for, that, soany, other, sug..."
5570,0,"[the, guy, did, some, bitching, but, acted, li..."


5) 불용어(stopwords) 제거

In [46]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [67]:
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
for i in range(data.shape[0]):
  tmp=[]
  for w in data['v2'][i]:
    if w not in stop_words:
      tmp.append(w)
  data['v2'][i]=tmp
data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,v1,v2
0,0,"[jurong, point, crazy, available, bugis, great..."
1,0,"[lar, joking, wif, oni]"
2,1,"[free, entry, wkly, comp, win, cup, final, tkt..."
3,0,"[dun, say, early, hor, already, say]"
4,0,"[nah, dont, think, goes, usf, lives, around, t..."
...,...,...
5567,1,"[2nd, time, tried, contact, the£750, pound, pr..."
5568,0,"[going, esplanade, home]"
5569,0,"[pity, mood, soany, suggestions]"
5570,0,"[guy, bitching, acted, like, interested, buyin..."


6) 표제어 추출(lemmatazition) 과정

In [49]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [68]:
from nltk.stem import WordNetLemmatizer
lem=WordNetLemmatizer()
for i in range(data.shape[0]):
  tmp=[]
  for w in data['v2'][i]:
    tmp.append(lem.lemmatize(w))
  data['v2'][i]=tmp
data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,v1,v2
0,0,"[jurong, point, crazy, available, bugis, great..."
1,0,"[lar, joking, wif, oni]"
2,1,"[free, entry, wkly, comp, win, cup, final, tkt..."
3,0,"[dun, say, early, hor, already, say]"
4,0,"[nah, dont, think, go, usf, life, around, though]"
...,...,...
5567,1,"[2nd, time, tried, contact, the£750, pound, pr..."
5568,0,"[going, esplanade, home]"
5569,0,"[pity, mood, soany, suggestion]"
5570,0,"[guy, bitching, acted, like, interested, buyin..."


데이터 분리

In [15]:
from sklearn.model_selection import train_test_split
train, test=train_test_split(data, test_size=0.2, random_state=30)

In [18]:
X_train=train['v2']
Y_train=train['v1']

In [21]:
X_train

353     Yo you guys ever figure out how much we need f...
3565    Its ok..come to my home it vl nice to meet and...
4335    Fyi I'm taking a quick shower, be at epsilon i...
3259    I'm always looking for an excuse to be in the ...
5044    We have sent JD for Customer Service cum Accou...
                              ...                        
919                 Dont talk to him ever ok its my word.
5260    If anyone calls for a treadmill say you'll buy...
500     Fighting with the world is easy, u either win ...
3885    Same, I'm at my great aunts anniversary party ...
4517    That sucks. So what do you got planned for you...
Name: v2, Length: 4457, dtype: object